# Spectroscopy Peaks


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/09_gallery_advanced/physics/spectroscopy_peaks.ipynb)

Advanced Spectroscopy Peak Fitting with fit() API and GlobalOptimizationConfig.

This example demonstrates fitting multiple peaks in a spectroscopy spectrum
using NLSQ's advanced fit() API and global optimization. Multi-peak fitting
is especially challenging due to many local minima, making global optimization
particularly important.

Compared to 04_gallery/physics/spectroscopy_peaks.py:
- Uses fit() instead of curve_fit() for automatic workflow selection
- Demonstrates GlobalOptimizationConfig for multi-start optimization
- Shows how presets ('robust', 'global') improve fitting reliability

Key Concepts:
- Multi-peak fitting (3 overlapping peaks)
- Gaussian and Lorentzian line shapes
- Linear background subtraction
- Peak area integration
- Global optimization for multi-modal landscapes (critical for spectroscopy)

In [ ]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("✅ NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

In [ ]:
import os
import sys
from pathlib import Path

import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

from nlsq import GlobalOptimizationConfig, fit

# Keep quick-mode runs light for CI/automation
QUICK_MODE = os.environ.get("NLSQ_EXAMPLES_QUICK") == "1"

# Set random seed
np.random.seed(42)

In [ ]:
def gaussian(x, amplitude, center, width):
    """Gaussian peak profile."""
    return amplitude * jnp.exp(-((x - center) ** 2) / (2 * width**2))


def lorentzian(x, amplitude, center, width):
    """Lorentzian peak profile (Cauchy distribution)."""
    return amplitude * width**2 / ((x - center) ** 2 + width**2)


def multi_peak_model(
    x,
    bg_slope,
    bg_offset,
    amp1,
    cen1,
    width1,  # Peak 1 (Gaussian)
    amp2,
    cen2,
    width2,  # Peak 2 (Gaussian)
    amp3,
    cen3,
    width3,
):  # Peak 3 (Lorentzian)
    """
    Model with 3 peaks (2 Gaussian + 1 Lorentzian) and linear background.

    Parameters
    ----------
    x : array_like
        Energy axis (keV)
    bg_slope, bg_offset : float
        Linear background parameters
    amp1, cen1, width1 : float
        Peak 1 parameters (Gaussian)
    amp2, cen2, width2 : float
        Peak 2 parameters (Gaussian)
    amp3, cen3, width3 : float
        Peak 3 parameters (Lorentzian)

    Returns
    -------
    y : array_like
        Total spectrum (background + peaks)
    """
    background = bg_slope * x + bg_offset
    peak1 = gaussian(x, amp1, cen1, width1)
    peak2 = gaussian(x, amp2, cen2, width2)
    peak3 = lorentzian(x, amp3, cen3, width3)
    return background + peak1 + peak2 + peak3


# Energy axis (keV for X-ray spectroscopy)
energy = np.linspace(5, 15, 80 if QUICK_MODE else 500)

# True parameters
bg_slope_true = 2.0
bg_offset_true = 50.0

# Peak 1: K-alpha line (Gaussian, strong)
amp1_true, cen1_true, width1_true = 800, 7.5, 0.3

# Peak 2: K-beta line (Gaussian, weaker, overlapping)
amp2_true, cen2_true, width2_true = 400, 8.5, 0.25

# Peak 3: Escape peak (Lorentzian, weak, broad)
amp3_true, cen3_true, width3_true = 200, 11.0, 0.4

# Generate true spectrum
spectrum_true = multi_peak_model(
    energy,
    bg_slope_true,
    bg_offset_true,
    amp1_true,
    cen1_true,
    width1_true,
    amp2_true,
    cen2_true,
    width2_true,
    amp3_true,
    cen3_true,
    width3_true,
)

# Add Poisson noise
noise = np.random.normal(0, np.sqrt(spectrum_true + 10), size=len(energy))
spectrum_measured = spectrum_true + noise

# Measurement uncertainties
sigma = np.sqrt(spectrum_measured + 10)


print("=" * 70)
print("SPECTROSCOPY PEAK FITTING: ADVANCED FITTING WITH fit() API")
print("=" * 70)
print("\nNote: Multi-peak fitting is challenging due to many local minima.")
print("Global optimization is critical for reliable parameter estimation.")

In [ ]:
# =============================================================================
# Multi-Peak Fitting

In [ ]:
# =============================================================================
print("\n" + "-" * 70)
print("MULTI-PEAK FITTING (3 overlapping peaks)")
print("-" * 70)

# Initial guess
p0 = [
    1.5,
    40,  # background
    750,
    7.5,
    0.4,  # peak 1
    350,
    8.5,
    0.3,  # peak 2
    180,
    11.0,
    0.5,  # peak 3
]

# Parameter bounds
bounds = (
    [0, 0, 0, 6, 0.1, 0, 7, 0.1, 0, 9, 0.1],
    [10, 100, 2000, 9, 1.0, 1000, 10, 1.0, 500, 13, 1.0],
)

if QUICK_MODE:
    print("Quick mode: skipping optimization runs; using true parameters.")
    popt_robust = np.array([
        bg_slope_true,
        bg_offset_true,
        amp1_true,
        cen1_true,
        width1_true,
        amp2_true,
        cen2_true,
        width2_true,
        amp3_true,
        cen3_true,
        width3_true,
    ])
    pcov_robust = np.eye(len(p0)) * 0.01
    popt_global, pcov_global = popt_robust, pcov_robust
    popt_custom, pcov_custom = popt_robust, pcov_robust
    perr_robust = np.sqrt(np.diag(pcov_robust))
    perr_global = np.sqrt(np.diag(pcov_global))
    perr_custom = np.sqrt(np.diag(pcov_custom))
else:
    # Method 1: fit() with 'robust' preset
    print("\nMethod 1: fit() with 'robust' preset")
    popt_robust, pcov_robust = fit(
        multi_peak_model,
        energy,
        spectrum_measured,
        p0=p0,
        sigma=sigma,
        bounds=bounds,
        absolute_sigma=True,
        preset="robust",
    )

    perr_robust = np.sqrt(np.diag(pcov_robust))

    # Extract parameters
    (
        bg_slope_r,
        bg_offset_r,
        amp1_r,
        cen1_r,
        width1_r,
        amp2_r,
        cen2_r,
        width2_r,
        amp3_r,
        cen3_r,
        width3_r,
    ) = popt_robust

    print("Peak Centers (robust):")
    print(f"  Peak 1 (K-alpha): {cen1_r:.3f} keV (true: {cen1_true})")
    print(f"  Peak 2 (K-beta):  {cen2_r:.3f} keV (true: {cen2_true})")
    print(f"  Peak 3 (Escape):  {cen3_r:.3f} keV (true: {cen3_true})")

    # Method 2: fit() with 'global' preset (CRITICAL for spectroscopy)
    global_starts = 2 if QUICK_MODE else 20
    print(f"\nMethod 2: fit() with 'global' preset ({global_starts} starts)")
    print("  (Global optimization is especially important for multi-peak fitting)")
    popt_global, pcov_global = fit(
        multi_peak_model,
        energy,
        spectrum_measured,
        p0=p0,
        sigma=sigma,
        bounds=bounds,
        absolute_sigma=True,
        preset="global",
        n_starts=global_starts,
    )

    perr_global = np.sqrt(np.diag(pcov_global))

    (
        bg_slope_g,
        bg_offset_g,
        amp1_g,
        cen1_g,
        width1_g,
        amp2_g,
        cen2_g,
        width2_g,
        amp3_g,
        cen3_g,
        width3_g,
    ) = popt_global

    print("Peak Centers (global):")
    print(f"  Peak 1 (K-alpha): {cen1_g:.3f} keV")
    print(f"  Peak 2 (K-beta):  {cen2_g:.3f} keV")
    print(f"  Peak 3 (Escape):  {cen3_g:.3f} keV")

    # Method 3: GlobalOptimizationConfig with many starts (for difficult spectra)
    custom_starts = 2 if QUICK_MODE else 30
    print(
        f"\nMethod 3: GlobalOptimizationConfig with {custom_starts} starts (thorough search)"
    )
    popt_custom, pcov_custom = fit(
        multi_peak_model,
        energy,
        spectrum_measured,
        p0=p0,
        sigma=sigma,
        bounds=bounds,
        absolute_sigma=True,
        multistart=True,
        n_starts=custom_starts,
        sampler="lhs",
    )

    perr_custom = np.sqrt(np.diag(pcov_custom))

    (
        bg_slope_c,
        bg_offset_c,
        amp1_c,
        cen1_c,
        width1_c,
        amp2_c,
        cen2_c,
        width2_c,
        amp3_c,
        cen3_c,
        width3_c,
    ) = popt_custom

    print("Peak Centers (30 starts):")
    print(f"  Peak 1 (K-alpha): {cen1_c:.3f} keV")
    print(f"  Peak 2 (K-beta):  {cen2_c:.3f} keV")
    print(f"  Peak 3 (Escape):  {cen3_c:.3f} keV")

# Use global preset results for detailed analysis (most reliable for spectroscopy)
popt = popt_global
perr = perr_global

(
    bg_slope_fit,
    bg_offset_fit,
    amp1_fit,
    cen1_fit,
    width1_fit,
    amp2_fit,
    cen2_fit,
    width2_fit,
    amp3_fit,
    cen3_fit,
    width3_fit,
) = popt

(
    bg_slope_err,
    bg_offset_err,
    amp1_err,
    cen1_err,
    width1_err,
    amp2_err,
    cen2_err,
    width2_err,
    amp3_err,
    cen3_err,
    width3_err,
) = perr


print("\n" + "=" * 70)
print("FITTED PARAMETERS (Global Preset - Recommended for Spectroscopy)")
print("=" * 70)

print("\nBackground:")
print(f"  Slope:  {bg_slope_fit:.3f} +/- {bg_slope_err:.3f}")
print(f"  Offset: {bg_offset_fit:.2f} +/- {bg_offset_err:.2f}")

print("\nPeak 1 (K-alpha, Gaussian):")
print(f"  Amplitude: {amp1_fit:.1f} +/- {amp1_err:.1f} counts")
print(f"  Center:    {cen1_fit:.3f} +/- {cen1_err:.3f} keV")
print(f"  Width:     {width1_fit:.3f} +/- {width1_err:.3f} keV")
print(f"  FWHM:      {2.355 * width1_fit:.3f} keV")
area1 = amp1_fit * width1_fit * np.sqrt(2 * np.pi)
print(f"  Area:      {area1:.0f} counts*keV")

print("\nPeak 2 (K-beta, Gaussian):")
print(f"  Amplitude: {amp2_fit:.1f} +/- {amp2_err:.1f} counts")
print(f"  Center:    {cen2_fit:.3f} +/- {cen2_err:.3f} keV")
print(f"  Width:     {width2_fit:.3f} +/- {width2_err:.3f} keV")
print(f"  FWHM:      {2.355 * width2_fit:.3f} keV")
area2 = amp2_fit * width2_fit * np.sqrt(2 * np.pi)
print(f"  Area:      {area2:.0f} counts*keV")

print("\nPeak 3 (Escape, Lorentzian):")
print(f"  Amplitude: {amp3_fit:.1f} +/- {amp3_err:.1f} counts")
print(f"  Center:    {cen3_fit:.3f} +/- {cen3_err:.3f} keV")
print(f"  Width:     {width3_fit:.3f} +/- {width3_err:.3f} keV (HWHM)")
print(f"  FWHM:      {2 * width3_fit:.3f} keV")
area3 = np.pi * amp3_fit * width3_fit
print(f"  Area:      {area3:.0f} counts*keV")

# Goodness of fit
chi_squared = np.sum(
    ((spectrum_measured - multi_peak_model(energy, *popt)) / sigma) ** 2
)
dof = len(energy) - len(popt)
chi_squared_reduced = chi_squared / dof

print("\nGoodness of Fit:")
print(f"  chi^2/dof = {chi_squared_reduced:.2f} (expect ~1.0)")


In [ ]:
# =============================================================================
# Peak Ratios

In [ ]:
# =============================================================================
print("\n" + "-" * 70)
print("PEAK RATIOS AND INTERPRETATION")
print("-" * 70)

print(f"Peak intensity ratio (K-alpha/K-beta): {area1 / area2:.2f}")
print("(Typical for many elements: ~1.5-2.5)")

energy_separation = abs(cen2_fit - cen1_fit)
print(f"K-alpha to K-beta separation: {energy_separation:.3f} keV")

In [ ]:
# =============================================================================
# Visualization